
## Introduction

Ce projet vise � comparer deux approches de machine learning - les r�seaux de neurones convolutifs (CNN) et les K-Nearest Neighbors (KNN) - pour d�tecter la pneumonie � partir d'images de radiographie. Nous �valuerons ces mod�les en fonction de leur pr�cision, de leur perte, et d'autres m�triques pertinentes, incluant les faux n�gatifs, faux positifs, et des graphiques tels que la matrice de confusion et la courbe ROC.

<style>
h1 {color: navy;}
</style>



### Structure du Dataset

Le dataset se compose de trois dossiers principaux :
- **train** : utilis� pour l'entra�nement des mod�les
- **val** : utilis� pour la validation des mod�les apr�s l'entra�nement
- **test** : utilis� pour tester les mod�les

Chaque dossier contient deux sous-dossiers :
- **NORMAL** : contient des images de radiographies normales
- **PNEUMONIA** : contient des images de radiographies avec pneumonie



### Filtrage des images avec ImageUtils

La classe `ImageUtils` est utilis�e pour filtrer les images de radiographie afin de s'assurer qu'elles respectent certaines dimensions minimales. Voici comment elle fonctionne :

```python
class ImageUtils:
    @staticmethod
    def filter_images(data_dir, img_size=(256, 256)):
        """
        Filter out images that are too small and gather statistics on the image sizes.
        """
        min_img_size = (img_size[0] * 2, img_size[1] * 2)
        image_stats = {
            'total_images': 0,
            'min_size': (float('inf'), float('inf')),
            'max_size': (0, 0),
            'avg_width': 0,
            'avg_height': 0,
            'filtered_images': 0
        }
        filepaths = []
        total_width, total_height = 0, 0

        for dirpath, _, filenames in os.walk(data_dir):
            for filename in filenames:
                if filename.lower().endswith(('png', 'jpg', 'jpeg')):
                    filepath = os.path.join(dirpath, filename)
                    with Image.open(filepath) as img:
                        width, height = img.size
                        image_stats['total_images'] += 1
                        total_width += width
                        total_height += height
                        if width < min_img_size[0] or height < min_img_size[1]:
                            image_stats['filtered_images'] += 1
                        else:
                            filepaths.append(filepath)
                            image_stats['min_size'] = (
                                min(image_stats['min_size'][0], width),
                                min(image_stats['min_size'][1], height)
                            )
                            image_stats['max_size'] = (
                                max(image_stats['max_size'][0], width),
                                max(image_stats['max_size'][1], height)
                            )
                            image_stats['avg_width'] += width
                            image_stats['avg_height'] += height

        remaining_images = image_stats['total_images'] - image_stats['filtered_images']
        if remaining_images > 0:
            image_stats['avg_width'] /= remaining_images
            image_stats['avg_height'] /= remaining_images

        image_stats['avg_width'] = round(image_stats['avg_width'])
        image_stats['avg_height'] = round(image_stats['avg_height'])

        filtered_filepaths = [fp for fp in filepaths if
                              Image.open(fp).size[0] >= min_img_size[0] and Image.open(fp).size[1] >=
                              min_img_size[1]]
        return filtered_filepaths, image_stats
```


In [ ]:

import json
import matplotlib.pyplot as plt
import numpy as np

# Chargement des r�sultats
with open("knn_results.json", "r") as file:
    knn_results = json.load(file)
with open("cnn_results.json", "r") as file:
    cnn_results = json.load(file)

# Affichage des r�sultats KNN
print("## R�sultats du mod�le KNN\n- Pr�cision: {}".format(knn_results['accuracy']))
print("- Rapport de classification:\n{}".format(knn_results['classification_report']))

# Affichage des r�sultats CNN
print("\n## R�sultats du mod�le CNN\n- Pr�cision de test: {}".format(cnn_results['test_accuracy']))
print("- Perte de test: {}".format(cnn_results['test_loss']))

# Donn�es pour la comparaison
labels = ['KNN', 'CNN']
accuracy = [knn_results['accuracy'], cnn_results['test_accuracy']]

# Cr�ation des graphiques
plt.figure(figsize=(8, 6))
plt.bar(labels, accuracy, color=['blue', 'green'])
plt.title('Comparaison de la pr�cision')
plt.ylabel('Pr�cision')
plt.show()



### Mod�le KNN

Le mod�le KNN (K-Nearest Neighbors) est un mod�le de machine learning simple mais efficace pour la classification. Voici comment il est construit et utilis� dans ce projet :

1. **Pr�traitement des images** : Les images sont redimensionn�es et normalis�es.
2. **Cr�ation du dataset** : Les images sont transform�es en vecteurs et divis�es en ensembles d'entra�nement et de validation.
3. **Entra�nement** : Le mod�le KNN est entra�n� sur les images d'entra�nement.
4. **�valuation** : Le mod�le est �valu� sur les images de validation pour calculer la pr�cision et d'autres m�triques.



# Graphique KNN
<style>
h1 {color: navy;}
</style>


In [ ]:

import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay

# Chargement des r�sultats
with open("knn_results.json", "r") as file:
    knn_results = json.load(file)

# Convertir la matrice de confusion en tableau NumPy
cm = np.array(knn_results['confusion_matrix'])

def plot_roc_curve(fpr, tpr, roc_auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(cm):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['NORMAL', 'PNEUMONIA'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

# Plot ROC curve
plot_roc_curve(knn_results['fpr'], knn_results['tpr'], knn_results['roc_auc'])

# Plot Confusion Matrix
plot_confusion_matrix(cm)



### Mod�le CNN

Le mod�le CNN (Convolutional Neural Network) est un mod�le de machine learning puissant pour la classification d'images. Voici comment il est construit et utilis� dans ce projet :

1. **Pr�traitement des images** : Les images sont redimensionn�es et normalis�es.
2. **Cr�ation du dataset** : Les images sont transform�es en vecteurs et divis�es en ensembles d'entra�nement et de validation.
3. **Entra�nement** : Le mod�le CNN est construit avec plusieurs couches de convolution, de pooling et de couches denses.
4. **�valuation** : Le mod�le est �valu� sur les images de validation pour calculer la pr�cision et d'autres m�triques.



# Graphique CNN
<style>
h1 {color: navy;}
</style>


In [ ]:

import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay

# Chargement des r�sultats
with open("cnn_results.json", "r") as file:
    cnn_results = json.load(file)

# Convertir la matrice de confusion en tableau NumPy
cm = np.array(cnn_results['confusion_matrix'])

def plot_roc_curve(fpr, tpr, roc_auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1

], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(cm):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['NORMAL', 'PNEUMONIA'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

# Plot ROC curve
plot_roc_curve(cnn_results['fpr'], cnn_results['tpr'], cnn_results['roc_auc'])

# Plot Confusion Matrix
plot_confusion_matrix(cm)



## Conclusion

En comparant les deux mod�les, nous observons que les deux approches ont des performances solides pour la d�tection de la pneumonie � partir d'images de radiographie. 

Le mod�le CNN montre une meilleure pr�cision et un score AUC ROC plus �lev�, ce qui indique une meilleure capacit� � distinguer entre les cas de pneumonie et les cas normaux. Cependant, le mod�le KNN reste une option valide et plus simple � impl�menter.

Pour des applications n�cessitant une haute pr�cision et la capacit� de traiter de grands ensembles de donn�es avec des images complexes, le mod�le CNN est recommand�. En revanche, pour des sc�narios o� les ressources informatiques sont limit�es et une impl�mentation rapide est n�cessaire, le mod�le KNN peut �tre plus appropri�.
